In [1]:
import itertools
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas
from tqdm.notebook import tqdm

warnings.simplefilter("ignore")

plt.style.use('seaborn')
plt.style.use('seaborn-notebook')

from soma.util.errors import compute_errors
from soma.generators.eye import EyeGenerator
from soma.tests.som import som_test

In [2]:
counts_per_label = EyeGenerator.count_per_label()
counts_per_label

{'R': 4262, 'I': 3804, 'C': 2870}

In [3]:
generators = dict(
    C=EyeGenerator('C'),
    I=EyeGenerator('I'),
    R=EyeGenerator('R'),
)

Comparing with

*A Fast and Effective Large-Scale Two-Sample Test Based on Kernels*

# p-values
As table 6, 0 is better (rejects they are equal)

In [4]:
pairwise = list(itertools.combinations(generators.keys(), 2))

In [5]:
results = {}
for a, b in pairwise:
    sample_size = min(counts_per_label[a], counts_per_label[b])
    results[f'{a} vs {b}'] = som_test(generators[a].sample(sample_size), generators[b].sample(sample_size))
results = pandas.DataFrame.from_dict(results, orient='index')
results

,0
C vs I,0.0
C vs R,0.0
I vs R,0.0


# Power of the test
As table 8, higher is better

In [8]:
sample_sizes = np.arange(1, 5) * 100
significance = 0.001
repeat = int(5 * (1 / significance))

## I vs C

In [9]:
results = {'error1': [], 'power': []}
for sample_size in tqdm(sample_sizes):
    error1, error2 = compute_errors(generators['I'], generators['C'], som_test, alpha=significance, samples=sample_size, repeat=repeat)
    results['error1'].append(error1)
    results['power'].append(1. - error2)
results = pandas.DataFrame.from_dict(results).set_index(sample_sizes)
results

  0%|          | 0/4 [00:00<?, ?it/s]

,error1,power
100,0.0,0.0432
200,0.0,0.9368
300,0.0,1.0000
400,0.0,1.0000


## I vs R

In [10]:
results = {'error1': [], 'power': []}
for sample_size in tqdm(sample_sizes):
    error1, error2 = compute_errors(generators['I'], generators['R'], som_test, alpha=significance, samples=sample_size, repeat=repeat)
    results['error1'].append(error1)
    results['power'].append(1. - error2)
results = pandas.DataFrame.from_dict(results).set_index(sample_sizes)
results

  0%|          | 0/4 [00:00<?, ?it/s]

,error1,power
100,0.0000,0.0000
200,0.0000,0.0018
300,0.0000,0.0078
400,0.0002,0.0248


## R vs C

In [11]:
results = {'error1': [], 'power': []}
for sample_size in tqdm(sample_sizes):
    error1, error2 = compute_errors(generators['R'], generators['C'], som_test, alpha=significance, samples=sample_size, repeat=repeat)
    results['error1'].append(error1)
    results['power'].append(1. - error2)
results = pandas.DataFrame.from_dict(results).set_index(sample_sizes)
results

  0%|          | 0/4 [00:00<?, ?it/s]

,error1,power
100,0.0000,0.0078
200,0.0000,0.6836
300,0.0000,0.9908
400,0.0004,1.0000


## I vs I

Obviously this must report a low power, since it should **not** be able to reject $H_0$ (both distributions are the same!).
This is a cross-check to verify the code is working as intended.

In [12]:
results = {'error1': [], 'power': []}
for sample_size in tqdm(sample_sizes):
    error1, error2 = compute_errors(generators['I'], generators['I'], som_test, alpha=significance, samples=sample_size, repeat=repeat)
    results['error1'].append(error1)
    results['power'].append(1. - error2)
results = pandas.DataFrame.from_dict(results).set_index(sample_sizes)
results

  0%|          | 0/4 [00:00<?, ?it/s]

,error1,power
100,0.0,0.0
200,0.0,0.0
300,0.0,0.0
400,0.0,0.0
